Tratar CSV

In [1]:
import datetime
import pandas as pd
import re

def timestamp_para_datahora(timestamp):
    datahora = datetime.datetime.fromtimestamp(timestamp)
    return datahora

import pandas as pd

def merge_dataframe(df, group_size=60):
    # Cria o dataframe auxiliar
    df_merged = pd.DataFrame()

    # Loop pelos grupos de tamanho "group_size"
    for i in range(0, len(df), group_size):
        # Seleciona o grupo atual
        df_group = df.iloc[i:i+group_size,:]

        # Loop pelas colunas do grupo
        for col in df_group.columns:

            # Verifica se a coluna é a datetime
            if col == 'datetime':
                # Armazena o valor da primeira linha da coluna
                col_value = df_group[col].iloc[0]

            else:
                # Armazena o valor da primeira linha da coluna
                col_value = df_group[col].iloc[0]

                # Loop pelas linhas da coluna
                for j in range(1, len(df_group)):

                    # Verifica se o valor atual é maior ou menor que o valor armazenado
                    if df_group[col].iloc[j] > col_value:
                        # Soma a diferença na variável armazenada
                        col_value += df_group[col].iloc[j] - col_value
                        
                    elif df_group[col].iloc[j] < col_value:
                        # Soma a diferença na variável armazenada
                        col_value -= col_value - df_group[col].iloc[j]

            # Insere a coluna e valor no dataframe auxiliar
            df_merged.loc[i//group_size, col] = col_value

    return df_merged



def formatar_dataframe(dataframe: pd.DataFrame) -> pd.DataFrame:
    # Converter a coluna "time" para valores numéricos e filtrar os valores nulos e não numéricos
    dataframe['time'] = pd.to_numeric(dataframe['time'], errors='coerce')
    dataframe = dataframe[~dataframe['time'].isna()]

    # Converter a coluna "time" para timestamp e criar a coluna "datetime"
    dataframe['datetime'] = dataframe['time'].astype(int).apply(timestamp_para_datahora)

    # Apagar a coluna "time", "summary", "icon", "cloudCover"
    dataframe = dataframe.drop(columns=['time', 'summary', 'icon', 'cloudCover'])

    # Ordena alfabeticamente as colunas
    dataframe = dataframe.sort_index(axis=1)

    return merge_dataframe(dataframe)

Salvar CSV Tratado

In [2]:
filepath = '../Files/HomeCTratado.csv'
dataframe = formatar_dataframe(pd.read_csv('../Files/HomeC.csv', delimiter=',', low_memory=False))

dataframe.to_csv(filepath, index=False)

C:\Users\Alan-\AppData\Local\Temp\ipykernel_17572\3347288179.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['datetime'] = dataframe['time'].astype(int).apply(timestamp_para_datahora)


Treinar modelo

In [5]:

X_train = formatar_dataframe(pd.read_csv('../Files/HomeCTratado.csv', delimiter=',', low_memory=False))

# Mostrar o resultado
X_train

C:\Users\Alan-\AppData\Local\Temp\ipykernel_16028\3436300967.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['datetime'] = dataframe['time'].astype(int).apply(timestamp_para_datahora)
c:\Users\Alan-\AppData\Local\Programs\Python\Python39\lib\site-packages\dask\dataframe\methods.py:349: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  df[name] = val


ValueError: cannot reindex on an axis with duplicate labels

Modelo

In [ ]:
from sklearn.ensemble import IsolationForest

# Treinando o modelo Isolation Forest
clf = IsolationForest(contamination=0.1, random_state=42)
clf.fit(X_train)

# Detectando anomalias nos dados de teste
y_pred = clf.predict(X_test) # -> Falta dados de teste